In [1]:
import json
import numpy as np
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1
print('ok')

ok


In [7]:
allgames = []
yeararray = [1952]
#yeararray = [1921, 1922, 1925, 1927, 1928]
#for i in np.linspace(1930,2016,87):
#    yeararray.append(i)

years_with_derived = []
for i in np.linspace(1942,1959,18):
    years_with_derived.append(i)
for i in np.linspace(1962,1968,7):
    years_with_derived.append(i)
for i in np.linspace(1971,1973,3):
    years_with_derived.append(i)
    
for thisyear in yeararray:
#    print('Starting to parse the year {0:.0f}'.format(thisyear))
    filename = 'raw/events_raw_{0:.0f}.json'.format(thisyear)
    print(filename)    
    with open(filename,'r') as f:
        games = json.load(f)
    if (thisyear in years_with_derived):
        derived_filename = 'with_derived/events_with_derived_{0:.0f}.json'.format(thisyear)
        print(derived_filename)
        with open(derived_filename,'r') as f:
            games_derived = json.load(f)
        games = games + games_derived
    allgames = allgames + games
    print('Data loaded for {0:.0f}: {1:,.0f} games'.format(thisyear, len(allgames)))
    print('...')
    
    column_list = ['gameID', 'event_in_game', 'event_type', 'inning', 'visitor_or_home', 'batter_id']
    column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_id', 'sub_batting_order', 'sub_position']#, 'gameID']
    
    #for i in (24,35,323,352,378):
    for i in range(0, len(allgames)):
        game_events_df = pandas.DataFrame.from_records(allgames[i]['events'])
        game_events_df['event_id'] = -1
        game_events_df['gameID'] = allgames[i]['id']
        if ((np.mod(i,100) == 0) | (i == len(allgames)-1)):
            print('Parsing game number {0:,.0f}: {1:}'.format(i, allgames[i]['id']))

        for varname in ('comment', 'sub_team', 'sub_id', 'sub_batting_order', 'sub_position'):
            if (varname not in game_events_df):                # if a game had no comments and/or substitutes, fill with blanks
                game_events_df[varname] = ''
        game_events_df = game_events_df[column_list]

        game_events_df['inning'] = pandas.to_numeric(game_events_df['inning'], errors='coerce')
        game_events_df['visitor_or_home'] = pandas.to_numeric(game_events_df['visitor_or_home'], errors='coerce')
        game_events_df['sub_team'] = pandas.to_numeric(game_events_df['sub_team'], errors='coerce')
        game_events_df['sub_batting_order'] = pandas.to_numeric(game_events_df['sub_batting_order'], errors='coerce')
        game_events_df['sub_position'] = pandas.to_numeric(game_events_df['sub_position'], errors='coerce')
        game_events_df = game_events_df.set_index('event_in_game')
        game_events_df = game_events_df.fillna('')

        current_comment = ''
        first_appearance = -1
        for eig, thisevent in game_events_df.iterrows():
            if (thisevent['event_type'] == 'com'):
    #            print(eig, thisevent['comment'])
                if (eig == 1):
                    game_events_df['event_type'].set_value(1, 'gameinfo')
                    first_appearance = 1
                elif (game_events_df['event_type'].loc[eig-1] != 'com'):
                    first_appearance = eig
                current_comment += ' ' + thisevent['comment']
            else:
                if (current_comment != ''):
                    current_comment = current_comment.replace('"','')
                    game_events_df['comment'].set_value(first_appearance-1, current_comment)
    #                print(first_appearance,current_comment,'\n')                
                current_comment = ''
                first_appearance = -1

        # do it again to take care of the case when the last event in a game is a comment
        if (game_events_df['event_type'].iloc[-1] == 'com'):
            current_comment = current_comment.replace('"','')
            game_events_df['comment'].set_value(first_appearance-1, current_comment)

        game_events_df = game_events_df[game_events_df['event_type'] != 'com']        

        game_gameinfo_df = game_events_df[game_events_df['event_type'] == 'gameinfo']
        game_events_df = game_events_df[game_events_df['event_type'] != 'gameinfo']

        # re-index now that we are missing rows (again, counting events in the game starting from 1)
        game_events_df = game_events_df.reset_index(drop=True)
        game_events_df['event_in_game'] = game_events_df.index.values+1
        game_events_df = game_events_df.set_index('event_in_game')
        
        if (len(game_gameinfo_df) > 0):
#            print('Inserting extra information in game {0:,.0f}, {1:}'.format(i, allgames[i]['id']))
            allgames[i]['info']['comment'] = game_gameinfo_df['comment'].values.tolist()[0]

        allgames[i]['events'] = game_events_df.values.tolist()
                                    
#    print('Writing files...')
#    gameinfo_df.to_csv('games/game_info_extra_{0:.0f}.csv'.format(thisyear))
#    events_df.to_csv('events_data/events_{0:.0f}.csv'.format(thisyear))

    print('Saving file...')
    with open('events_json/events_{0:.0f}.json'.format(thisyear),'w') as f:
        json.dump(allgames, f)

print('Done')

raw/events_raw_1952.json
with_derived/events_with_derived_1952.json
Data loaded for 1952: 1,199 games
...
Parsing game number 0: BSN195204150
Parsing game number 100: SLA195206221
Parsing game number 200: BOS195209241
Parsing game number 300: CHN195205270
Parsing game number 400: CIN195207140
Parsing game number 500: CLE195209110
Parsing game number 600: BRO195206170
Parsing game number 700: WS1195207190
Parsing game number 800: NYA195208290
Parsing game number 900: PHI195205272
Parsing game number 1,000: PIT195208241
Parsing game number 1,100: SLN195206070
Parsing game number 1,198: CHN195208172
Saving file...
Done


In [ ]:
column_list = ['event_id', 'gameID', 'event_in_game', 'event_type', 'inning', 'visitor_or_home', 'batter_id']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_id', 'sub_batting_order', 'sub_position']#, 'gameID']

events_df = pandas.DataFrame(columns=column_list)

#for i in range(0, len(allgames)):
global_count_offset = 0
#for i in (323, 352, 378):
for i in range(0,36):
#for i in range(0, len(allgames)):
    game_events_df = pandas.DataFrame.from_records(allgames[i]['events'])
    game_events_df['event_id'] = -1
    game_events_df['gameID'] = allgames[i]['id']
    for varname in ('comment', 'sub_team', 'sub_id', 'sub_batting_order', 'sub_position'):
        if (varname not in game_events_df):                # if a game had no comments and/or substitutes, fill with blanks
            game_events_df[varname] = ''
    game_events_df = game_events_df[column_list]

    game_events_df['inning'] = pandas.to_numeric(game_events_df['inning'], errors='coerce')
    game_events_df['visitor_or_home'] = pandas.to_numeric(game_events_df['visitor_or_home'], errors='coerce')
    game_events_df['sub_team'] = pandas.to_numeric(game_events_df['sub_team'], errors='coerce')
    game_events_df['sub_batting_order'] = pandas.to_numeric(game_events_df['sub_batting_order'], errors='coerce')
    game_events_df['sub_position'] = pandas.to_numeric(game_events_df['sub_position'], errors='coerce')
    game_events_df = game_events_df.set_index('event_in_game')
    game_events_df = game_events_df.fillna('')
    
    if (np.mod(i, 10) == 0):
        print('Parsing game number {0:,.0f}, containing {1:,.0f} events.'.format(i, len(game_events_df)))
    
    # NOTE: this all takes iloc and so goes off of event_in_game, which is a 1-based counter
#    print(game_events_df.tail(1))

    if (game_events_df['event_type'].iloc[1] == 'com'):
        print('COMMENT IN FIRST LINE in game number {0:,.0f}!!!!!!!!!!!!!!!'.format(i))
#        full_comment = game_events_df['comment'].iloc[1] + ' ' + game_events_df['comment'].iloc[2]
#        game_events_df['comment'].set_value(2, full_comment)       
        
    for j in range(len(game_events_df)-1, -2, -1):   # iterate from last to first: be sure to do something if the first event is a comment
        if (game_events_df['event_type'].iloc[j] == 'com'):
#            print(j, game_events_df.iloc[j])
            full_comment = game_events_df['comment'].iloc[j]
            if (j < len(game_events_df)-1):
                full_comment = full_comment + ' ' + game_events_df['comment'].iloc[j+1]
#            if (full_comment[0] == '"'):
#                full_comment = full_comment[1:]
#            if (full_comment[-1] == '"'):
#                full_comment = full_comment[:-1]
            if (game_events_df['comment'].iloc[j-1] != ''):
                game_events_df['comment'].set_value(j, full_comment)
                
#        print(game_events_df.iloc[1])
        
    # drop comment rows
    game_events_df = game_events_df[game_events_df['event_type'] != 'com']

    # re-index now that we are missing rows (again, counting events in the game starting from 1)
    game_events_df = game_events_df.reset_index(drop=True)
    game_events_df['event_in_game'] = game_events_df.index.values+1
    #game_events_df = game_events_df.set_index('event_in_game')

    game_events_df['event_id'] = game_events_df['event_in_game'] + global_count_offset
   
    events_df = pandas.concat((events_df, game_events_df))
    
    global_count_offset += len(game_events_df)    



events_df = events_df[column_list]
events_df = events_df.set_index('event_id')
#events_df.to_csv('events/events_1921.csv')
#events_df
events_df[events_df['comment'] != '']

#print('Done')

In [ ]:
column_list = ['event_id', 'gameID', 'event_in_game', 'event_type', 'inning', 'visitor_or_home', 'batter_id']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_id', 'sub_batting_order', 'sub_position']#, 'gameID']

events_df = pandas.DataFrame(columns=column_list)
#events_df = events_df.set_index('event_id')

globalcount = 0
for i in range(0, len(allgames)):
    if (np.mod(i, 100) == 0):
        print(i)
    game_events_df = pandas.DataFrame.from_records(allgames[i]['events'])
    game_events_df['event_id'] = globalcount + game_events_df['event_in_game']
    game_events_df['gameID'] = allgames[i]['id']


    
    game_events_df = game_events_df[column_list]
#    game_events_df = game_events_df.set_index('event_in_game')

    #game_events_df[game_events_df['event_type'] == 'comment'].head(5)
    
    events_df = pandas.concat((events_df, game_events_df))
    globalcount = globalcount + len(game_events_df)

events_df['event_in_game'] = pandas.to_numeric(events_df['event_in_game'], errors='coerce')
events_df = events_df.set_index('event_id', drop=True)
#events_df = events_df.drop('event_in_game', axis=1)
print('Done')

In [ ]:
events_df['inning'] = pandas.to_numeric(events_df['inning'], errors='coerce')
events_df['visitor_or_home'] = pandas.to_numeric(events_df['visitor_or_home'], errors='coerce')
events_df['sub_team'] = pandas.to_numeric(events_df['sub_team'], errors='coerce')
events_df['sub_batting_order'] = pandas.to_numeric(events_df['sub_batting_order'], errors='coerce')
events_df['sub_position'] = pandas.to_numeric(events_df['sub_position'], errors='coerce')
events_df = events_df.fillna('')          
print('ok')

In [ ]:
events_df
#events_df[events_df['event_type'] == 'com']

In [ ]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_batting_order', 'sub_position']#, 'gameID']

for i in range(0,1):
#for i in range(0,len(allgames)):
    thisgamedict = allgames[i]
    this_game_events_df = pandas.DataFrame(thisgamedict['events'], columns=column_list)
    this_game_events_df = this_game_events_df.set_index('event_in_game')
this_game_events_df[this_game_events_df['comment'] != '']

In [ ]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_batting_order', 'sub_position']#, 'gameID']

show_columns_for_weirdness = ['event_in_game', 'inning', 'home_or_visitor', 'event', 'comment']

for i in range(0,len(allgames)):
    thisgamedict = allgames[i]
    thisgameevents_df = pandas.DataFrame(thisgamedict['events'], columns=column_list)
    if (numpy.mod(i,100) == 0):
        print('processing game {0:,.0f}...'.format(i))
    for j in range(0,len(thisgameevents_df)): #- 1):
        if (thisgameevents_df.iloc[j]['event_type'] != 'play'):
            thisgameevents_df.iloc[j]['inning'] = thisgameevents_df.iloc[j-1]['inning']
            thisgameevents_df.iloc[j]['home_or_visitor'] = thisgameevents_df.iloc[j-1]['home_or_visitor']
            
        if (thisgameevents_df.iloc[j-1]['event_type'] not in ('play','sub')):
            thisgameevents_df.iloc[j]['comment'] = thisgameevents_df.iloc[j-1]['comment'] + ' ' + thisgameevents_df.iloc[j]['comment']

    thisgameevents_df = thisgameevents_df[(thisgameevents_df['event_type'] == 'play') | (thisgameevents_df['event_type'] == 'sub')]
    thisgameevents_df = thisgameevents_df.drop('event_in_game',axis=1)
    thisgameevents_df = thisgameevents_df.reset_index(drop=True)
    thisgameevents_df.index.name = 'event_in_game'
    allgames[i]['events'] = thisgameevents_df.reset_index().values.tolist()

print('preparing to save file...')
#thisgameevents_df
with open('events_json/events_1930.json','w') as f:
    json.dump(allgames, f)
print('done')

In [ ]:
    # replace event types with number codes for fastness
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'play'),0,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'sub'),1,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'com'),2,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'badj'),3,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'padj'),4,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'badj'),5,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = pandas.to_numeric(thisgameevents_df['event_type'])
    
